#### Importiamo le librerie necessarie. Useremo ResNet50.

In [182]:
from tensorflow import keras
import numpy as np
import tarfile
from time import time
from tensorflow.keras.applications import resnet50, vgg19, mobilenet_v2, xception
import matplotlib.pyplot as plt
import pandas as pd
import datetime, os
import pathlib
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications import inception_v3
from IPython import display
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Creiamo train e test set

## Prima configurazione

In [2]:
train_dir = "data_mel/train" 
val_dir = "data_mel/val"
test_dir = "data_mel/test"

In [3]:
species = np.array(tf.io.gfile.listdir(str(train_dir)))
species = species[species != 'README.md']
n_classes= len(species)
print('Species:', species)

Species: ['Alces_alces' 'Bos_taurus' 'Cervus_elaphus' 'Equus_caballus'
 'Lutra_lutra' 'Ovis' 'Pan' 'Panthera_leo' 'Procyon' 'Vulpes']


In [183]:
train_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function= xception.preprocess_input)
                                                                #shear_range=0.2,
                                                                #zoom_range=0.2,
                                                                #horizontal_flip=True,
                                                                #rotation_range=20,
                                                                #width_shift_range=0.2,
                                                                #height_shift_range=0.2,
                                                                #brightness_range=[0.5, 1.5])


train_generator = train_processing.flow_from_directory(directory = train_dir,
                                    target_size = (224, 224),
                                    color_mode = "rgb",
                                    batch_size = 32,
                                    class_mode = "categorical",
                                    shuffle = True,
                                    seed = 1234)

Found 916 images belonging to 10 classes.


#### Ora andiamo a definire il test generator, in cui non applicheremo alcun tipo di data agumentation ovviamente, in quanto i dati di test devono essere utilizzati come sono.

In [184]:
val_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=xception.preprocess_input)

val_generator = val_processing.flow_from_directory(
        directory=val_dir,
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

Found 119 images belonging to 10 classes.


In [185]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=xception.preprocess_input)

test_generator = test_processing.flow_from_directory(
        directory=test_dir,
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

Found 148 images belonging to 10 classes.


#### Importiamo il modello con i pesi adatti (imagenet), e settiamo i layer iniziali come non allenabili

In [186]:
base_net = xception.Xception(weights="imagenet", include_top=False,
	input_shape=(224, 224, 3), pooling="avg")

83689472/83683744 [==============================] - 18s 0us/step


In [187]:
for layer in base_net.layers:
    layer.trainable = False

In [188]:
base_net.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_10[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

#### Ora aggiungiamo due fully connected layers e poi creiamo un nuovo modello "net", dato dalla combinazione dei due 

In [189]:
# Output of the base_net model
x = base_net.output#get_layer('flatten').output
# intermediate fully-connected layer + ReLU
#x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)
# final fully-connected layer + SoftMax 
pred = keras.layers.Dense(n_classes, activation='softmax')(x)

In [190]:
model1 = keras.Model(inputs=base_net.input, outputs=pred)

In [191]:
model2 = keras.Sequential(
    [
        keras.layers.Conv2D(8, kernel_size=(3,3), activation = 'relu'),
        keras.layers.Conv2D(8, kernel_size=(3,3), activation = 'relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(16, kernel_size=(3,3), activation = 'relu'),
        keras.layers.Conv2D(16, kernel_size=(3,3), activation = 'relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(32, kernel_size=(3,3), activation = 'relu'),
        keras.layers.Conv2D(32, kernel_size=(3,3), activation = 'relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, kernel_size=(3,3), activation = 'relu'),
        keras.layers.Conv2D(64, kernel_size=(3,3), activation = 'relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.BatchNormalization(),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ]
)

In [192]:
model=model1

In [193]:
#net1.summary()

#### Compiliamo il modello usando come loss la categorical crossentropy, come optimizer RMSprop e come metrica l'accuracy

In [194]:
model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adam(lr=0.0005),
            metrics=['accuracy'])

#### Ora fittiamo il modello con 10 epoche e visualizziamo i risultati in termini di accuracy e loss su train e test set

In [195]:
cp_callback = keras.callbacks.ModelCheckpoint(filepath="monodim_weights",
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor="val_accuracy",
                                                 verbose=1)


In [196]:
history1 = model.fit(train_generator,
          epochs=50,
          validation_data=val_generator,
          callbacks=[cp_callback])

Epoch 1/50
29/29 [==============================] - 12s 275ms/step - loss: 2.3035 - accuracy: 0.1597 - val_loss: 2.0256 - val_accuracy: 0.2605

Epoch 00001: val_accuracy improved from -inf to 0.26050, saving model to monodim_weights
Epoch 2/50
29/29 [==============================] - 6s 199ms/step - loss: 2.0290 - accuracy: 0.2835 - val_loss: 1.8555 - val_accuracy: 0.3950

Epoch 00002: val_accuracy improved from 0.26050 to 0.39496, saving model to monodim_weights
Epoch 3/50
29/29 [==============================] - 6s 201ms/step - loss: 1.8138 - accuracy: 0.3844 - val_loss: 1.7377 - val_accuracy: 0.4370

Epoch 00003: val_accuracy improved from 0.39496 to 0.43697, saving model to monodim_weights
Epoch 4/50
29/29 [==============================] - 6s 200ms/step - loss: 1.6660 - accuracy: 0.4508 - val_loss: 1.6430 - val_accuracy: 0.4790

Epoch 00004: val_accuracy improved from 0.43697 to 0.47899, saving model to monodim_weights
Epoch 5/50
29/29 [==============================] - 6s 200ms/s

In [ ]:
#Open tensorboard
#Open localhost:6006

In [ ]:
def accuracy_loss(history):
    plt.figure(figsize=(20,10))
    #
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label = "train_accuracy", color = "red")
    plt.plot(history.history['val_accuracy'], label="train_accuracy", color ="blue")
    plt.legend(loc='best', fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.xlabel('Epochs', size = 15)
    plt.ylabel('Accuracy', size = 15)
    #
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label = "Train loss", color = "red")
    plt.plot(history.history['val_loss'], label="Test loss", color = "blue")
    plt.legend(loc='best', fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.xlabel('Epochs', size = 15)
    plt.ylabel('Loss', size = 15)

In [ ]:
monodim_model = net1.to_json()

with open("monodim_model.json", "w") as json_file:
    json_file.write(monodim_model)
    
load_status = net1.load_weights("monodim_weights")
load_status.assert_consumed()

In [ ]:
performance1 = net1.evaluate(test_generator)

In [ ]:
accuracy_loss(history1)